In [1]:
"""將PTT目標版面的文章、關鍵字標題、希望爬取頁數,將網址download至'pttFile.csv'檔案中"""

def pttURL_crawler(board,keyWd,pageNo,writeFile):
    import requests
    from bs4 import BeautifulSoup
    import csv

    """擷取ptt文章列表的網頁,擷取每篇文章的連結
       並翻每一頁網頁,爬取想要的頁數"""

    URL = "https://www.ptt.cc/"
    headers = {"cookies" : "over18=1"}
    #希望爬取ptt板面、但是頁面擷取的網址內容會缺乏URL,需要自行補上
    res = requests.get(URL + "bbs/" + board +"/index.html" , headers = headers)
    soup = BeautifulSoup(res.text , 'lxml')

    bottons = soup.select('a.btn.wide')
    #totalpage=上一頁的頁數+1
    totalpage = int(bottons[1]['href'].split('index')[1].split('.')[0])+1

    """在希望的頁數（totalpage ~ crawlpage-n)的文章列表中,希望爬出文章標題為"KeyWord"的網址
       所以,只要爬到該文章就將網址寫入'pttFile.csv'
    """
    
    for page in range(totalpage,totalpage - pageNo, -1):
    
        res = requests.get(URL+ "bbs/" + board +"/index{}.html".format(page),headers = headers)    
        soup = BeautifulSoup(res.text,'lxml')
        links = soup.select('div.title > a')
        
        #爬取該頁面中每一篇有'keyword'的文章
        for link in links:
            title = link.text
            if(keyWd) in title:        
                article_URL = URL +link['href']
                print(article_URL) 
                #只要爬到該keyword的文章寫入'pttFile.csv'
                with open(writeFile,'a') as fw:
                    fw.write(article_URL + '\n')
    
    return print("download finished!")


In [2]:
"""將csv檔案中的網址（ptt）內容爬出（限定只有英文的部份）並寫入檔案中"""

def pttEng_crawler(readFile,writeFile):
    import requests
    from bs4 import BeautifulSoup
    import csv
    import re

    """#將檔案中的網址讀取,並進入該網址（ptt）的頁面"""
    with open(readFile,'r') as fr:
        for URL in fr:
            article_URL = URL.strip()        
            res   = requests.get(article_URL,headers = {"cookies" : "over18=1"})
            soup  = BeautifulSoup(res.text,"lxml") 
            links = soup.select("div.article-metaline > span")
            
            #預期將所有文章的英文串接
            """將每篇文章的內容爬出,並過濾爬出只有英文字的內容,每篇文章的英文字會以'*'串接,並寫入檔案中"""
            for link in links:

                words = []    
                article_dict ={}   
                #去除文章本文以外的標籤    
                trash_link = soup.select('div.article-metaline > span') + soup.select('div.article-metaline-right > span') + soup.select('div.push') + soup.select('div#article-polling') 
                for trash in trash_link:
                    trash.decompose()
                #過濾文章內文,並將有英文的部份全部改為小寫    
                content = soup.select_one("#main-container")
                for script in content.find_all('script'):
                    script.decompose()
                for a in content.select('a'):
                    a.decompose()
                article_dict['content'] =content.text
                content = article_dict['content'].lower()
                #過濾文章內容的特殊符號

                clearContent = content.replace('\n', '').replace('(', '').replace(')', '').replace(',', '').replace('.','')
                #限定文章中只有英文是我們想要的內容
                word = list(set(re.findall('[A-Za-z]+[+#-C]*',clearContent, re.IGNORECASE)))
                #該篇文章所有的英文字串接為words
    #                 print(word)
                words += word

            with open(writeFile,'a') as fw:
                fw.write('*'.join(words) + '\n')
                
    return print("get the English part of these article!")         

In [3]:
"""將想要比較（本文文章）和被比較的檔案（有keyWord的斷詞）,相互比較得到keyWord出現的次數"""

def CountKeyWd(compareFile,comparedFile,result):
    from collections import Counter
    import re
    #open 被比較的檔案（含有keyWord的斷詞）
    with open(comparedFile,'r') as f:
        lan = [text.lower() for text in f.read().split('\n')]
    #open 想要比較的本文檔案  
    with open(compareFile,'r') as fr:        
        wc = Counter()
        """只要本文檔案,每讀取一行（words)就經過處理成為好幾個word,
        並判斷該word是否有在被比較檔案中（含有keyWord斷詞）,
        如果有就value+1,反之就value為1"""
        for strWd in fr:
            words  = strWd.strip().split('*')       

            for word in words:
                if word in lan:
                    if word in wc:
                        wc[word] += 1
                    else: 
                        wc[word] = 1        
    """計算出各關鍵字出現數量就將檔案寫入'result'csv檔案中"""
    with open(result, "w") as fw:
        for k,v in  wc.most_common():
            fw.write('{},{}'.format(k,v)  + '\n')
    return print(wc.most_common())  


In [ ]:
pttURL_crawler("Soft_Job","徵才",1247,"softURL.csv")
pttEng_crawler("softURL.csv","softEng.csv")
CountKeyWd('softEng.csv','lan.csv','newSoftCount.csv')

https://www.ptt.cc//bbs/Soft_Job/M.1493893370.A.3B7.html
https://www.ptt.cc//bbs/Soft_Job/M.1493965127.A.4F6.html
https://www.ptt.cc//bbs/Soft_Job/M.1493970262.A.C73.html
https://www.ptt.cc//bbs/Soft_Job/M.1493992126.A.B4E.html
https://www.ptt.cc//bbs/Soft_Job/M.1453196050.A.D41.html
https://www.ptt.cc//bbs/Soft_Job/M.1493714611.A.86B.html
https://www.ptt.cc//bbs/Soft_Job/M.1493650502.A.93A.html
https://www.ptt.cc//bbs/Soft_Job/M.1493710638.A.D87.html
https://www.ptt.cc//bbs/Soft_Job/M.1493711189.A.C32.html
https://www.ptt.cc//bbs/Soft_Job/M.1493372232.A.0A4.html
https://www.ptt.cc//bbs/Soft_Job/M.1493372976.A.094.html
https://www.ptt.cc//bbs/Soft_Job/M.1493474343.A.886.html
https://www.ptt.cc//bbs/Soft_Job/M.1493290122.A.22B.html
https://www.ptt.cc//bbs/Soft_Job/M.1493291205.A.A98.html
https://www.ptt.cc//bbs/Soft_Job/M.1493347162.A.FF8.html
https://www.ptt.cc//bbs/Soft_Job/M.1493009448.A.069.html
https://www.ptt.cc//bbs/Soft_Job/M.1493031693.A.AF0.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1486052847.A.BC9.html
https://www.ptt.cc//bbs/Soft_Job/M.1485842329.A.6DC.html
https://www.ptt.cc//bbs/Soft_Job/M.1486005164.A.393.html
https://www.ptt.cc//bbs/Soft_Job/M.1486010251.A.044.html
https://www.ptt.cc//bbs/Soft_Job/M.1485320449.A.716.html
https://www.ptt.cc//bbs/Soft_Job/M.1485159976.A.921.html
https://www.ptt.cc//bbs/Soft_Job/M.1484801134.A.F65.html
https://www.ptt.cc//bbs/Soft_Job/M.1484801666.A.843.html
https://www.ptt.cc//bbs/Soft_Job/M.1484829103.A.B97.html
https://www.ptt.cc//bbs/Soft_Job/M.1484830101.A.B60.html
https://www.ptt.cc//bbs/Soft_Job/M.1484830313.A.36F.html
https://www.ptt.cc//bbs/Soft_Job/M.1484830427.A.A5C.html
https://www.ptt.cc//bbs/Soft_Job/M.1484899453.A.816.html
https://www.ptt.cc//bbs/Soft_Job/M.1484661508.A.0BE.html
https://www.ptt.cc//bbs/Soft_Job/M.1484709894.A.A90.html
https://www.ptt.cc//bbs/Soft_Job/M.1484727965.A.407.html
https://www.ptt.cc//bbs/Soft_Job/M.1484740237.A.ECA.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1474342684.A.3BA.html
https://www.ptt.cc//bbs/Soft_Job/M.1474471511.A.1C5.html
https://www.ptt.cc//bbs/Soft_Job/M.1474508971.A.45E.html
https://www.ptt.cc//bbs/Soft_Job/M.1474512880.A.585.html
https://www.ptt.cc//bbs/Soft_Job/M.1474529861.A.57D.html
https://www.ptt.cc//bbs/Soft_Job/M.1473827741.A.6CC.html
https://www.ptt.cc//bbs/Soft_Job/M.1473851951.A.A6F.html
https://www.ptt.cc//bbs/Soft_Job/M.1473852123.A.824.html
https://www.ptt.cc//bbs/Soft_Job/M.1474278970.A.790.html
https://www.ptt.cc//bbs/Soft_Job/M.1473489967.A.B95.html
https://www.ptt.cc//bbs/Soft_Job/M.1473501446.A.3C0.html
https://www.ptt.cc//bbs/Soft_Job/M.1473501518.A.E56.html
https://www.ptt.cc//bbs/Soft_Job/M.1473739205.A.2C2.html
https://www.ptt.cc//bbs/Soft_Job/M.1473322608.A.C61.html
https://www.ptt.cc//bbs/Soft_Job/M.1473084340.A.582.html
https://www.ptt.cc//bbs/Soft_Job/M.1473130381.A.6E1.html
https://www.ptt.cc//bbs/Soft_Job/M.1473133892.A.FFC.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1463727383.A.D67.html
https://www.ptt.cc//bbs/Soft_Job/M.1463728635.A.440.html
https://www.ptt.cc//bbs/Soft_Job/M.1463936388.A.6A7.html
https://www.ptt.cc//bbs/Soft_Job/M.1463491824.A.3F6.html
https://www.ptt.cc//bbs/Soft_Job/M.1463634451.A.540.html
https://www.ptt.cc//bbs/Soft_Job/M.1463654464.A.8BD.html
https://www.ptt.cc//bbs/Soft_Job/M.1463211339.A.75D.html
https://www.ptt.cc//bbs/Soft_Job/M.1463288922.A.EEE.html
https://www.ptt.cc//bbs/Soft_Job/M.1463361497.A.858.html
https://www.ptt.cc//bbs/Soft_Job/M.1463459485.A.F7A.html
https://www.ptt.cc//bbs/Soft_Job/M.1463068334.A.841.html
https://www.ptt.cc//bbs/Soft_Job/M.1463107721.A.DCA.html
https://www.ptt.cc//bbs/Soft_Job/M.1463114768.A.8C7.html
https://www.ptt.cc//bbs/Soft_Job/M.1463199363.A.ACC.html
https://www.ptt.cc//bbs/Soft_Job/M.1462889050.A.446.html
https://www.ptt.cc//bbs/Soft_Job/M.1462893380.A.81C.html
https://www.ptt.cc//bbs/Soft_Job/M.1462897413.A.0E6.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1452567581.A.55D.html
https://www.ptt.cc//bbs/Soft_Job/M.1452004200.A.BA9.html
https://www.ptt.cc//bbs/Soft_Job/M.1451845995.A.D71.html
https://www.ptt.cc//bbs/Soft_Job/M.1451886166.A.764.html
https://www.ptt.cc//bbs/Soft_Job/M.1451665750.A.BA9.html
https://www.ptt.cc//bbs/Soft_Job/M.1451704247.A.25E.html
https://www.ptt.cc//bbs/Soft_Job/M.1451713501.A.02A.html
https://www.ptt.cc//bbs/Soft_Job/M.1451587675.A.A38.html
https://www.ptt.cc//bbs/Soft_Job/M.1451620890.A.AAA.html
https://www.ptt.cc//bbs/Soft_Job/M.1451662657.A.DFE.html
https://www.ptt.cc//bbs/Soft_Job/M.1451400722.A.87D.html
https://www.ptt.cc//bbs/Soft_Job/M.1451402041.A.D3C.html
https://www.ptt.cc//bbs/Soft_Job/M.1451403483.A.FC3.html
https://www.ptt.cc//bbs/Soft_Job/M.1451404534.A.1D4.html
https://www.ptt.cc//bbs/Soft_Job/M.1451412869.A.689.html
https://www.ptt.cc//bbs/Soft_Job/M.1451417630.A.DFF.html
https://www.ptt.cc//bbs/Soft_Job/M.1451436585.A.B06.html
https://www.ptt.cc//bbs/Soft_Jo

...

https://www.ptt.cc//bbs/Soft_Job/M.1301123392.A.9ED.html
https://www.ptt.cc//bbs/Soft_Job/M.1300790812.A.8FE.html
https://www.ptt.cc//bbs/Soft_Job/M.1300876352.A.B00.html
https://www.ptt.cc//bbs/Soft_Job/M.1300945821.A.8D6.html
https://www.ptt.cc//bbs/Soft_Job/M.1300755472.A.782.html
https://www.ptt.cc//bbs/Soft_Job/M.1300775985.A.6A7.html
https://www.ptt.cc//bbs/Soft_Job/M.1300351498.A.373.html
https://www.ptt.cc//bbs/Soft_Job/M.1300411050.A.1EB.html
https://www.ptt.cc//bbs/Soft_Job/M.1300199403.A.1FA.html
https://www.ptt.cc//bbs/Soft_Job/M.1300282138.A.D2B.html
https://www.ptt.cc//bbs/Soft_Job/M.1299841189.A.1EB.html
https://www.ptt.cc//bbs/Soft_Job/M.1299953577.A.5CB.html
https://www.ptt.cc//bbs/Soft_Job/M.1300067291.A.999.html
https://www.ptt.cc//bbs/Soft_Job/M.1300155600.A.120.html
https://www.ptt.cc//bbs/Soft_Job/M.1299636553.A.FF8.html
https://www.ptt.cc//bbs/Soft_Job/M.1299656745.A.34C.html
https://www.ptt.cc//bbs/Soft_Job/M.1299657603.A.E4C.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1280061271.A.399.html
https://www.ptt.cc//bbs/Soft_Job/M.1279685417.A.2B1.html
https://www.ptt.cc//bbs/Soft_Job/M.1279546732.A.95B.html
https://www.ptt.cc//bbs/Soft_Job/M.1279265995.A.ED0.html
https://www.ptt.cc//bbs/Soft_Job/M.1279292114.A.413.html
https://www.ptt.cc//bbs/Soft_Job/M.1279390878.A.5F6.html
https://www.ptt.cc//bbs/Soft_Job/M.1279018758.A.670.html
https://www.ptt.cc//bbs/Soft_Job/M.1279028488.A.3DD.html
https://www.ptt.cc//bbs/Soft_Job/M.1279125367.A.799.html
https://www.ptt.cc//bbs/Soft_Job/M.1279192420.A.316.html
https://www.ptt.cc//bbs/Soft_Job/M.1279205437.A.B67.html
https://www.ptt.cc//bbs/Soft_Job/M.1278837353.A.979.html
https://www.ptt.cc//bbs/Soft_Job/M.1278465936.A.523.html
https://www.ptt.cc//bbs/Soft_Job/M.1278571067.A.DB3.html
https://www.ptt.cc//bbs/Soft_Job/M.1278583608.A.4F4.html
https://www.ptt.cc//bbs/Soft_Job/M.1278040131.A.D4E.html
https://www.ptt.cc//bbs/Soft_Job/M.1278095850.A.493.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1256088080.A.4FC.html
https://www.ptt.cc//bbs/Soft_Job/M.1256108962.A.4C1.html
https://www.ptt.cc//bbs/Soft_Job/M.1255672639.A.AF5.html
https://www.ptt.cc//bbs/Soft_Job/M.1255013265.A.8AD.html
https://www.ptt.cc//bbs/Soft_Job/M.1255338982.A.279.html
https://www.ptt.cc//bbs/Soft_Job/M.1254659520.A.F3B.html
https://www.ptt.cc//bbs/Soft_Job/M.1253949056.A.D7F.html
https://www.ptt.cc//bbs/Soft_Job/M.1253965852.A.D7B.html
https://www.ptt.cc//bbs/Soft_Job/M.1254285924.A.C83.html
https://www.ptt.cc//bbs/Soft_Job/M.1253781068.A.7C1.html
https://www.ptt.cc//bbs/Soft_Job/M.1253794520.A.3C8.html
https://www.ptt.cc//bbs/Soft_Job/M.1253531808.A.89A.html
https://www.ptt.cc//bbs/Soft_Job/M.1253588383.A.DC6.html
https://www.ptt.cc//bbs/Soft_Job/M.1253163293.A.54E.html
https://www.ptt.cc//bbs/Soft_Job/M.1253255426.A.BE4.html
https://www.ptt.cc//bbs/Soft_Job/M.1252505604.A.A28.html
https://www.ptt.cc//bbs/Soft_Job/M.1252655461.A.158.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1236744934.A.0A3.html
https://www.ptt.cc//bbs/Soft_Job/M.1236756109.A.344.html
https://www.ptt.cc//bbs/Soft_Job/M.1236838328.A.B9C.html
https://www.ptt.cc//bbs/Soft_Job/M.1236495565.A.135.html
https://www.ptt.cc//bbs/Soft_Job/M.1236005607.A.0D5.html
https://www.ptt.cc//bbs/Soft_Job/M.1236138855.A.452.html
https://www.ptt.cc//bbs/Soft_Job/M.1235892870.A.EA0.html
https://www.ptt.cc//bbs/Soft_Job/M.1235397919.A.063.html
https://www.ptt.cc//bbs/Soft_Job/M.1235460392.A.3CD.html
https://www.ptt.cc//bbs/Soft_Job/M.1235578759.A.E6D.html
https://www.ptt.cc//bbs/Soft_Job/M.1235054093.A.A06.html
https://www.ptt.cc//bbs/Soft_Job/M.1235112294.A.1CC.html
https://www.ptt.cc//bbs/Soft_Job/M.1235367205.A.27B.html
https://www.ptt.cc//bbs/Soft_Job/M.1234665532.A.35C.html
https://www.ptt.cc//bbs/Soft_Job/M.1234697917.A.E84.html
https://www.ptt.cc//bbs/Soft_Job/M.1234759628.A.B9A.html
https://www.ptt.cc//bbs/Soft_Job/M.1234834724.A.C21.html
https://www.ptt.cc//bbs/Soft_Jo

https://www.ptt.cc//bbs/Soft_Job/M.1204127522.A.A45.html
https://www.ptt.cc//bbs/Soft_Job/M.1204272398.A.295.html
https://www.ptt.cc//bbs/Soft_Job/M.1204354775.A.7FF.html
https://www.ptt.cc//bbs/Soft_Job/M.1203861251.A.677.html
https://www.ptt.cc//bbs/Soft_Job/M.1203942817.A.DB0.html
https://www.ptt.cc//bbs/Soft_Job/M.1203491567.A.545.html
https://www.ptt.cc//bbs/Soft_Job/M.1203519451.A.4C9.html
https://www.ptt.cc//bbs/Soft_Job/M.1203319482.A.E13.html
https://www.ptt.cc//bbs/Soft_Job/M.1203040414.A.269.html
https://www.ptt.cc//bbs/Soft_Job/M.1203067824.A.CCE.html
https://www.ptt.cc//bbs/Soft_Job/M.1202866224.A.2E5.html
https://www.ptt.cc//bbs/Soft_Job/M.1202898829.A.070.html
https://www.ptt.cc//bbs/Soft_Job/M.1202965071.A.BE3.html
https://www.ptt.cc//bbs/Soft_Job/M.1201760231.A.ED3.html
https://www.ptt.cc//bbs/Soft_Job/M.1201230837.A.D12.html
https://www.ptt.cc//bbs/Soft_Job/M.1201068516.A.7DB.html
https://www.ptt.cc//bbs/Soft_Job/M.1200381295.A.7EF.html
https://www.ptt.cc//bbs/Soft_Jo

In [4]:
pttEng_crawler("pttTech_file.csv","techEng.csv")
CountKeyWd('techEng.csv','lan.csv','newTechCount.csv')

get the English part of these article!
[('', 392), ('linux', 177), ('java', 164), ('android', 154), ('c', 134), ('ios', 114), ('c++', 107), ('javascript', 104), ('php', 102), ('project', 84), ('mysql', 78), ('python', 77), ('c#', 74), ('html', 70), ('jquery', 64), ('ruby', 58), ('matlab', 38), ('word', 37), ('excel', 36), ('security', 30), ('objective-c', 26), ('r', 26), ('ajax', 25), ('oracle', 25), ('unix', 24), ('fpga', 23), ('vmware', 21), ('rf', 20), ('shell', 19), ('drivers', 18), ('xml', 18), ('illustrator', 17), ('http', 17), ('base', 16), ('powerpoint', 15), ('flash', 15), ('verilog', 14), ('access', 14), ('spring', 14), ('dsp', 14), ('arm', 13), ('perl', 13), ('dns', 13), ('labview', 13), ('mcu', 12), ('eda', 12), ('autocad', 11), ('wan', 11), ('firmware', 10), ('bluetooth', 9), ('iis', 9), ('udp', 9), ('sap', 9), ('ideas', 9), ('swift', 8), ('solidworks', 8), ('vpn', 8), ('apple', 7), ('xhtml', 7), ('firewall', 7), ('san', 7), ('rdbms', 7), ('mes', 7), ('socket', 7), ('sas',